In [ ]:
import lob

import numpy as np, pandas as pd
import matplotlib as mpl, matplotlib.pyplot as plt
import tqdm
import importlib
importlib.reload(lob)

<module 'lob' from 'c:\\Users\\ebryce\\OneDrive\\Documents\\Projects\\LOB-Model\\microstructure\\lob.py'>

In [ ]:
importlib.reload(lob)

market = lob.Market(randomize_environment=True)

books = {}
mids = []
bids = []
offers = []
trades = {}
iters = 100
for i in tqdm.tqdm(range(iters)):
    trade = market.lob.send(market._random_order(fair_value=market.lob.get_mid(), marketable=True))
    market.lob.send(market._random_order(fair_value=market.lob.get_mid(), marketable=False))
    book = market.lob.get_level_2_book()
    books[i] = book
    mids.append(market.lob.get_mid())
    bids.append(market.lob.get_bid())
    offers.append(market.lob.get_offer())
    trades[i] = trade
books = pd.concat(books)
books = books.stack().unstack(1).unstack(1).dropna(axis=1)

mids, bids, offers = pd.Series({i: mids[i] for i in range(iters)}), pd.Series({i: bids[i] for i in range(iters)}), pd.Series({i: offers[i] for i in range(iters)})

plt.style.use('classic')
plt.rc('font', **{'family': 'Arial', 'weight': 'normal', 'size': 12})
fig = plt.figure(figsize=(7,5))
fig.patch.set_facecolor('white')
ax = fig.add_subplot(1, 1, 1)
ax.spines['left'].set_visible(True)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.grid(True,axis='both',linestyle=':')

ax.plot(mids, label='mid', ds='steps-post')
ax.plot(bids, label='bid', ds='steps-post')
ax.plot(offers, label='offer', ds='steps-post')

for t in trades:
    for price, qty in trades[t]:
        ax.scatter(x=t, y=price, s=qty, c='k', alpha=0.5)

ax.legend(frameon=False, fontsize='small')

print(trades)
plt.show()

print(books.iloc[-1].unstack().T.sort_index(ascending=False))


INFO:root:Randomizing the order book with 15 orders around 10.00; spread 0.05-0.15
DEBUG:root:No offer available, pricing at fair value 10.0000 plus half the minimum spread 0.05
INFO:root:Random passive order to S 200 at $10.02 joining nan (mkt=nan/nan)
INFO:root:Order to S 200 @ 10.02 not fully filled; posting 200 residual
DEBUG:root:Added order for 200 to offer level 10.02 (size at level now 200)
INFO:root:Random passive order to S 400 at $10.05 joining nan (mkt=nan/10.02)
INFO:root:Order to S 400 @ 10.05 not fully filled; posting 400 residual
DEBUG:root:Added order for 400 to offer level 10.05 (size at level now 400)
INFO:root:Random passive order to S 100 at $10.09 joining nan (mkt=nan/10.02)
INFO:root:Order to S 100 @ 10.09 not fully filled; posting 100 residual
DEBUG:root:Added order for 100 to offer level 10.09 (size at level now 100)
INFO:root:Random passive order to S 100 at $10.11 joining nan (mkt=nan/10.02)
INFO:root:Order to S 100 @ 10.11 not fully filled; posting 100 resid